# Trabajo Final Sistemas de Bases de datos Masivos

Con el presente trabajo pretendemos dar a conocer los resultados de    las encuestas realizadas por la comunidad de desarrolladores *Stackoverflow*, desde el año 2011 al 2017, además de una proyección de los resultados una posible encuesta en 2018.    Presentaremos desde la obtención y limpieza de los datos, pasando por    el analisis de tendencias, hasta la actualización de indicadores    mediante rutinas en diferentes herramientas que nos permiten poner en    practica los conocimientos adquiridos en la presente asignatura.

## 1. Obtención y Limpieza de los datos

### Obtención

Los datos se obtuvieron mediante la herramienta **BigQuery** donde se encuentran disponibles las preguntas de las encuestas como datos abiertos. En total son 7 conjuntos de datos que se guardaron en un conjunto de datos propios con el total de registros y columnas de cada año mediante la siguiente consulta:

In [ ]:
CREATE TABLE `Clase2_seccion2.survey_2011` 
AS
SELECT * FROM `fh-bigquery.stackoverflow.survey_results_2011`


Para saber que tipo de preguntas habian en las encuestas se hizo la siguiente rutina de **Python** que permitia buscar campos repetidos en los 7 conjuntos de datos

## Adjuntar rutina de mapeo de preguntas*

De esta forma se determinaron las preguntas que van a hacer parte de la bodega de datos

|Pregunta    | 2011 || 2012 || 2013 || 2014 || 2015 || 2016 || 2017 |
|-------------|------||------||------||------||------||------||------|
| Salario     |   Y  ||   Y  ||   Y  ||   Y  ||   Y  ||   Y  ||   Y  |
| Region      |   Y  ||   Y  ||   Y  ||   Y  ||   Y  ||   Y  ||   Y  |
| Lenguajes   |   Y  ||   Y  ||   Y  ||   Y  ||   Y  ||   Y  ||   Y  |
| Satisfacción|   Y  ||   Y  ||   Y  ||   N  ||   Y  ||   Y  ||   Y  |
| Género      |   N  ||   N  ||   N  ||   Y  ||   Y  ||   Y  ||   Y  |
| Os          |   Y  ||   Y  ||   Y  ||   Y  ||   Y  ||   Y  ||   Y  |
| Edad        |   Y  ||   Y  ||   Y  ||   Y  ||   Y  ||   Y  ||   N  |
| Experiencia |   Y  ||   Y  ||   Y  ||   Y  ||   Y  ||   Y  ||   Y  |
| Company Size|   Y  ||   Y  ||   Y  ||   Y  ||   N  ||   Y  ||   Y  |

Con las preguntas ya mapeadas, en **BigQuery** se hizo el respectivo filtro de las columnas necesarias con la cantidad total de filas y se exportó la información en formatos *JSON* y *CSV*

    SELECT 
    YearsProgram,Country,Salary,HaveWorkedLanguage,JobSatisfaction,Gender,HaveWorkedPlatform,companySize
    FROM `clase1-seccion2.Clase2_seccion2.survey_2017`

### Limpieza

Para la limpieza se uso una rutina en **Python** tomando los archivos en formato *JSON*

## Adjuntar rutinas **

Mediante las anteriores rutinas se generan nuevos archivos *JSON* que harán parte de la bodega de datos

# 2. Bodega de Datos

Con los 7 conjuntos de datos y el conjunto adicional que contiene todos los años anteriores, se crea la bodega de datos en **MongoDB**

 ![alt text](C/Documents/bodega.png)

Se genera la bodega de datos mediante las siguientes lineas de código

# Adjuntar linea de codigo**

Mediante la bodega de datos podemos responder las siguientes preguntas:

1. Cuál es la edad de los programadores senior en Alemania?

Se considera senior un ingeniero que tenga al menos seis años de experiencia


In [ ]:
db.data.aggregate([ {$match: {"experience_range" : {$in: ["6-10","10-15","15-20", "20-90"]} , 'region': 'Germany'}}, {$group: { _id: '$age_range' }} ])

In [ ]:
##Espacio para gráfica...

2. Cuáles son los 5 paises que mas tienen programadores?

    db.data.aggregate([
        {$group: {_id: '$region', total: {$sum: 1}}},
        {$sort: {'total':-1}},
        {$limit: 6}
    ])

In [7]:
#Espacio para gráfica....

3. Cuál es el Top de los 5 lenguajes más populares por año?

In [ ]:
#Adjuntar script 3 

In [8]:
#Espacio para gráfica

4. Cuál es el sistema operativo que usan los programadores de C#?

    df=pd.read_json('clean_files/data.json')
    df1=df[['os','programming_languages']]
    df2=df1[df1['programming_languages'].str.contains('C#', regex=False, na=False)]
    df2.dropna()
    splitlista=[]
    for i in df2['os']:
        if i is not None:
            for  e in i:
                splitlista.append(e)
    cuenta = collections.Counter(splitlista)
    print(cuenta.most_common(10))

In [9]:
#Espacio para gráficar

5. Cuál es el rango de edad de los programadores junior?

    db.data.aggregate([ {$match: {"experience_range" : {$in: ["0-2"]} }}, {$group: { _id: '$age_range' }} ])

In [10]:
#Espacio para gráfica

6. Cuantas mujeres programan en python?

    db.data.find({"programming_languages" : {$in: ["python", "Python"]}, 'gender': {$in: ['Female', 'female']}}).count()

In [11]:
#Espacio para gráfica

7. En qué lenguajes se sienten más insatisfechos los programadores?

In [ ]:
df2=df[['satisfaction','programming_languages']]
df3=df2[df2['satisfaction']==1]
splitlista=[]
for i in df3['programming_languages']:
    if i is not None:
        for  e in i:
            splitlista.append(e)

splitlista
minuscula=[]
[minuscula.append(i.lower()) for i in splitlista]
lista = [row.split(',') for row in minuscula]
lista
import itertools
from operator import itemgetter
conteo=[[key, len(list(group))] for key, group in itertools.groupby(sorted(lista, key=itemgetter(0)), itemgetter(0))]
conteo.sort(key=lambda x: x[-1])
print (conteo[-5:])

In [13]:
#Espacio para gráfica

8. Cuál es el rango de salario de los programadores de Java?

9. Definir...

10. Cuántas personas trabajaban en Start up en 2017?

Se asume que una startup tiene menos de 25 empleados

    db.survey2017.find({'company_size_range': '1-25'}).count()

In [ ]:
#Espacio para gráfica

11. Evolución del número de programadores por tamaño de la compañia

     var s11 = db.survey2011.aggregate([ {$group: {_id: '$company_size_range', total: {$sum: 1}}} ])
     var s12 = db.survey2012.aggregate([ {$group: {_id: '$company_size_range', total: {$sum: 1}}} ])
     var s13 = db.survey2013.aggregate([ {$group: {_id: '$company_size_range', total: {$sum: 1}}} ])
     var s14 = db.survey2014.aggregate([ {$group: {_id: '$company_size_range', total: {$sum: 1}}} ])
     var s15 = db.survey2015.aggregate([ {$group: {_id: '$company_size_range', total: {$sum: 1}}} ])
     var s16 = db.survey2016.aggregate([ {$group: {_id: '$company_size_range', total: {$sum: 1}}} ])
     var s17 = db.survey2017.aggregate([ {$group: {_id: '$company_size_range', total: {$sum: 1}}} ])

In [15]:
#Espacio para gráfica

12. Cuál es el lenguaje que utilizan las grandes compañias?

13. Numero de programadores por lenguaje en 2017

14. En que pais se paga mejor por programar en python?

    db.data.aggregate([ {$match: {'programming_languages': {$in: ['Python', 'python']}, 'salary_range': '>140k'}}, {$group: {_id: '$region'}}])

In [17]:
#Espacio para gráfica

15. Cual es el promedio de lenguajes que los programadores han utilizado?

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_json('queries/clean_files/data.json')

df['programming_languages_lenght'] = df['programming_languages'].apply(lambda x: len(x) if x is not None else None)
df['programming_languages_lenght'].mean()

In [ ]:
#Espacio para gráfica